In [167]:
from pysgx import stocks, info, util
import importlib
import pandas as pd

In [204]:
importlib.reload(info);
importlib.reload(stocks);

In [205]:
stock = stocks.load("D8DU")
for i in range(2000, 2024):
    start_day = str(i) + "-01-01"
    end_day = str(i+1) + "-01-01"
    print(i, stocks.get_risk(start_day, end_day, stock))

2000 ByAdjClose: nan ByAddBackClose: nan
2001 ByAdjClose: nan ByAddBackClose: nan
2002 ByAdjClose: nan ByAddBackClose: nan
2003 ByAdjClose: nan ByAddBackClose: nan
2004 ByAdjClose: nan ByAddBackClose: nan
2005 ByAdjClose: nan ByAddBackClose: nan
2006 ByAdjClose: nan ByAddBackClose: nan
2007 ByAdjClose: 0.000212 ByAddBackClose: 0.000212
2008 ByAdjClose: 0.001040 ByAddBackClose: 0.000838
2009 ByAdjClose: 0.000849 ByAddBackClose: 0.000782
2010 ByAdjClose: 0.000276 ByAddBackClose: 0.000266
2011 ByAdjClose: 0.000309 ByAddBackClose: 0.000279
2012 ByAdjClose: 0.000551 ByAddBackClose: 0.000544
2013 ByAdjClose: 0.000735 ByAddBackClose: 0.000735
2014 ByAdjClose: 0.000977 ByAddBackClose: 0.000977
2015 ByAdjClose: 0.000546 ByAddBackClose: 0.000546
2016 ByAdjClose: 0.000329 ByAddBackClose: 0.000329
2017 ByAdjClose: 0.000810 ByAddBackClose: 0.000810
2018 ByAdjClose: 0.000888 ByAddBackClose: 0.000888
2019 ByAdjClose: 0.000768 ByAddBackClose: 0.000768
2020 ByAdjClose: 0.006283 ByAddBackClose: 0.005535

# Adj Close return vs AddBack Return

In [183]:
# stock = stocks.load("D05")
stock = stocks.load("D8DU")
df = stocks.get_addback_close(stock, "2022-01-01", "2024-01-01")

return_adj_close = df["Adj Close"].pct_change().dropna()
return_addback_close = df["AddBackClose"].pct_change().dropna()

print(return_addback_close.mean())
print(return_adj_close.mean())
return_adj_close.std()  / return_addback_close.std() - 1
# return_adj_close.mean()  / return_addback_close.mean() - 1

-0.0002463114195503293
-0.00044110314428133615


0.04505055363062538

In [195]:
df = pd.concat([return_addback_close, return_adj_close], axis=1)
df.cov().iloc[0, 1]

0.000718800466679042

# Stock Market Cap Distribution

In [ ]:
df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
df.MarketCap = df.MarketCap / 1000000
plt = df[df.MarketCap<100].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.legend(['Total 365 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M.png')

plt = df[df.MarketCap>100][df.MarketCap<1500].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 100M~1.5B SGD)')
plt.legend(['Total 188 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M_1.5B.png')

plt = df[df.MarketCap>1500][df.MarketCap<30000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 1.5B~30B SGD)')
plt.legend(['Total 62 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_1.5B_20B.png')

df[df.MarketCap>10000].sort_values(by=['MarketCap']).to_csv("mc_10B_plus.csv", index=False, float_format='%.6f')

# Average Daily Trading Volume Hist

In [ ]:
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000

plt = df[df.AvergeVolume<=1].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) - Below(10K SGD)')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 365 stocks'])
plt.get_figure().savefig('adtv_10K.png')

plt = df[1<df.AvergeVolume][df.AvergeVolume<=30].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 10K~300K SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 207 stocks'])
plt.get_figure().savefig('adtv_10K_300K.png')

plt = df[df.AvergeVolume>30][df.AvergeVolume<=300].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 300K~3M SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 68 stocks'])
plt.get_figure().savefig('adtv_300K_3M.png')

df[df.AvergeVolume>300].sort_values(by=['AvergeVolume']).to_csv("adtv_3M_plus.csv", index=False, float_format='%.6f')

# STI Estimate

In [ ]:
es3 = stocks.load("ES3")
df = stocks.get_addback_close(es3, "2010-03-01", "2024-03-15")
df["Adj Close"] = df["Adj Close"] * 990
df["AddBackClose"] = df["AddBackClose"] * 1010
df["Close"] = df["Close"] * 1000

plt = df.plot(x="Day", y = ["AddBackClose", "Close", "Adj Close"], linewidth=0.8, figsize=(15, 10), title="Esitmated STI")
plt.legend(['Estimated STI if add back dividend', 'STI', 'Estimated STI if use Adj Close'])
plt.get_figure().savefig('STI_estimated.png')